In [1]:
import networkx as nx
import math
import time
import progressbar
import tqdm
import random
import collections
import matplotlib.pyplot as plt
%matplotlib inline
from multiprocessing import Pool, cpu_count, Queue, Manager
%reload_ext Cython
import numpy as np

In [8]:
tree_depth = 20
log_N = 10
N = 2 ** log_N

subtree_depth = tree_depth - log_N
threshold = tree_depth

In [9]:
def generate_label(i):
    label = []
    j = 0
    l = random.randint(1, 2 ** (tree_depth) - 1)
    depths[i] = int(l.bit_length() - 1)
    labels[i] = int(l)
    return label

# The first axis stores the depth of the node in the tree and the second axis stores the full label
depths = np.ndarray(shape=(N), dtype=np.uint32)
labels = np.ndarray(shape=(N), dtype=np.uint32)
for i in range(N):
    generate_label(i)

In [7]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
# --annotate
# distutils: language = c++

cimport numpy as np
from __main__ import threshold, tqdm, Manager, random
from __main__ import labels as c_labels
from __main__ import depths as c_depths
from __main__ import tree_depth as tree_depth
from __main__ import subtree_depth as subtree_depth

from libcpp.map cimport map
from libcpp.pair cimport pair
from libcpp.vector cimport vector

# __builtin_clz returns the number of leading zeros in the binary representation
cdef extern int __builtin_clz (unsigned int x)

ctypedef map['unsigned int', vector['unsigned int']] MapLabelToLabelVector

cdef unsigned int label_distance(unsigned int first_label, unsigned int second_label):
    cdef unsigned int dist = 0
    cdef unsigned int d1, l1, d2, l2, j
    cdef unsigned int height_delta
    cdef unsigned int first_label_depth = 32 - __builtin_clz(first_label)
    cdef unsigned int second_label_depth = 32 - __builtin_clz(second_label)
    
    if second_label_depth > first_label_depth:
        height_delta = second_label_depth - first_label_depth
        l1 = first_label
        l2 = second_label
    else:
        height_delta = first_label_depth - second_label_depth
        l1 = second_label
        l2 = first_label
    l2 = l2 >> height_delta
    dist = 2 * (31 ^ __builtin_clz(((l2 ^ l1) << 1) + 1)) + height_delta
    return dist

def compute_neighborhood_of_vertex(int i):
    cdef np.ndarray[unsigned int, ndim=1, mode="c"] labels = c_labels
    cdef np.ndarray[unsigned int, ndim=1, mode="c"] depths = c_depths
    cdef unsigned int *labels_ptr = &labels[0]
    cdef unsigned int *depths_ptr = &depths[0]
    cdef unsigned int num_labels = len(labels)
    cdef unsigned int j
    cdef unsigned int degree = 0
    cdef vector[pair['unsigned int', 'unsigned int']] edges
    
    for j in range(i+1, num_labels):
        dist = label_distance(labels[i], labels[j])
        if dist < threshold:
            edges.push_back(pair['unsigned int', 'unsigned int'](labels[i], labels[j]))
            degree += 1
        
    return edges

cdef unsigned int get_subtree_index(unsigned int label, unsigned int label_depth, unsigned int subtree_depth):
    cdef unsigned int extra_depth = label_depth % subtree_depth
    return label >> extra_depth

cdef pair[MapLabelToLabelVector, vector['unsigned int']] create_subtree_partitions(
    unsigned int *labels, unsigned int *depths, unsigned int num_labels):
    cdef unsigned int current_depth = 0
    cdef unsigned int extra_depth
    
    cdef MapLabelToLabelVector subtree_label_to_label_index_list
    cdef vector['unsigned int'] empty_vector
    cdef vector['unsigned int'] subtree_labels
    
    while current_depth < tree_depth:
        for partial_label in range(1 << current_depth):
            label = partial_label + (1 << current_depth)
            subtree_label_to_label_index_list[label] = empty_vector
            subtree_labels.push_back(label)
        current_depth += subtree_depth

    for i in range(num_labels):
        label = labels[i]
        depth = depths[i]
        extra_depth = depth % subtree_depth
        subtree_label_to_label_index_list[label >> extra_depth].push_back(label)
    
    return pair[MapLabelToLabelVector, vector['unsigned int']](
        subtree_label_to_label_index_list, subtree_labels)

def get_neighborhood_of_vertex(int I):
    cdef np.ndarray[unsigned int, ndim=1, mode="c"] labels = c_labels
    cdef np.ndarray[unsigned int, ndim=1, mode="c"] depths = c_depths
    cdef unsigned int *labels_ptr = &labels[0]
    cdef unsigned int *depths_ptr = &depths[0]
    cdef unsigned int num_labels = len(labels)
    cdef unsigned int i, j
    cdef unsigned int degree = 0
    cdef vector[pair['unsigned int', 'unsigned int']] edges
    
    cdef unsigned label = labels[0]
    cdef pair[MapLabelToLabelVector, vector['unsigned int']] X = create_subtree_partitions(
        labels_ptr, depths_ptr, num_labels)
    cdef MapLabelToLabelVector subtree_label_to_label_index_list = X.first
    cdef vector['unsigned int'] subtree_labels = X.second
    cdef unsigned int num_subtrees = subtree_labels.size()
    cdef vector['unsigned int'] first_subtree_member_labels, second_subtree_member_labels
    
    # Change this back
    for i in range( num_subtrees):
    # if True:
        first_subtree_label = subtree_labels[i]
        for j in range(i, num_subtrees):
            second_subtree_label = subtree_labels[j]
            dist = label_distance(first_subtree_label, second_subtree_label)
            if (dist < threshold):
                first_subtree_member_labels = subtree_label_to_label_index_list[first_subtree_label]
                second_subtree_member_labels = subtree_label_to_label_index_list[second_subtree_label]
                degree += first_subtree_member_labels.size() * second_subtree_member_labels.size()
                for first_endpoint in first_subtree_member_labels:
                    for second_endpoint in second_subtree_member_labels:
                        edges.push_back(pair['unsigned int', 'unsigned int'](first_endpoint, second_endpoint))
                # print(dist, first_subtree_member_labels.size(), second_subtree_member_labels.size())
    
    return edges

In [5]:
start_time = time.time()
edges = get_neighborhood_of_vertex(3330)
elapsed_time = time.time() - start_time
print("--- %s edges sampled ---" % len(edges))
print("--- %s nanoseconds per edge ---" % (1e9 * elapsed_time / len(edges)))

--- 264200 edges sampled ---
--- 351.66650175777556 nanoseconds per edge ---


In [10]:
num_iters = 4096
start_time = time.time()
correct_edges = []
degree = 0
for i in range(num_iters):
    correct_edges.extend(compute_neighborhood_of_vertex(i))
elapsed_time = time.time() - start_time
print("--- %s edges sampled ---" % len(correct_edges))
print("--- %s nanoseconds per edge ---" % (1e9 * elapsed_time / len(correct_edges)))

--- 1942 edges sampled ---
--- 5988.701487422358 nanoseconds per edge ---


In [107]:
optimized_edge_set = set([tuple(sorted((u, v))) for (u, v) in edges if u != v])
unoptimized_edge_set = set([tuple(sorted((u, v))) for (u, v) in correct_edges if u != v])

In [108]:
[edge for edge in optimized_edge_set if edge not in unoptimized_edge_set]

[(8, 13), (8, 44), (8, 40), (13, 56), (40, 44)]

In [109]:
[edge for edge in unoptimized_edge_set if edge not in optimized_edge_set]

[]

In [82]:
sorted(labels)

[3, 7, 8, 8, 11, 13, 16, 24, 26, 28, 40, 41, 42, 43, 53, 54]